# Using Financial Data Example #1: Calculating Altman Z" Score

Professor Altman first formulated his infamous "Z Score" in [1968](http://www.defaultrisk.com/_pdf6j4/Financial_Ratios_Discriminant_Anlss_n_Prdctn_o_Crprt_Bnkrptc.pdf) while at NYU. The "Z Score" attempts to quantify the likelihood that a company defaults. After several iterations, the Altman Z" (Double Prime) Score was developed to better quantify a company's credit risk. Professor Altman's presentation [here](http://pages.stern.nyu.edu/~ealtman/3-%20CopCrScoringModels.pdf) walks through several models including this one. When it was initially developed there were some crude cutoffs for the scores - above 2.6 and the firm was "healthy", between 1.1-2.6 was the "grey area", and below 1.1 and the firm as at risk of bankruptcy. However, over time that crude scale was refined. One of the unique things about the Altman Z" Score today is that we have a mapping to conventional credit ratings. Below we walk through the example of calculating the score from scratch using [IEX data](https://iextrading.com/developer/docs/).

$$ Z” = 6.56x_1 +3.26x_2 + 6.72x_3 + 1.05x_4 $$
$$ \textrm{Where:} $$
$$ x_1 = \textrm{Working Capital / Total Assets} $$
$$ x_2 = \textrm{Retained Earnings / Total Assets} $$
$$ x_3 = \textrm{EBIT / Total Assets} $$
$$ x_4 = \textrm{Market Value of Equity / Total Liabilities} $$

In [ ]:
import pyEX as p
c = p.Client(api_token="pk_353fe2ce67cd4c16b30a748ff783c865", version="v1")

In [ ]:
ticker = "aapl"
incomeStatement = c.incomeStatementDF(ticker)
balanceSheet = c.balanceSheetDF(ticker)
cfStatement = c.cashFlowDF(ticker)
stats = c.keyStats(ticker)

In [ ]:
x1 = ( balanceSheet["currentAssets"][0] - balanceSheet["totalCurrentLiabilities"][0] ) / balanceSheet["totalAssets"][0]

In [ ]:
x2 = balanceSheet["retainedEarnings"][0] / balanceSheet["totalAssets"][0]

In [ ]:
x3 = incomeStatement["ebit"][0] / balanceSheet["totalAssets"][0]

In [ ]:
x4 = stats["marketcap"] / balanceSheet["totalLiabilities"][0]

In [ ]:
6.56 * x1 + 3.26 * x2 + 6.72 * x3 + 1.05 * x4

In [ ]:
def altmanZDoublePrime( ticker ):
    '''
    Calculate the Altman Z" Score for a given ticker
    
    ticker = string, user input for which to calculate the Z-score. Not case sensitive.
    '''
    incomeStatement = c.incomeStatementDF(ticker)
    balanceSheet = c.balanceSheetDF(ticker)
    cfStatement = c.cashFlowDF(ticker)
    stats = c.keyStats(ticker)
    x1 = ( balanceSheet["currentAssets"][0] - balanceSheet["totalCurrentLiabilities"][0] ) / balanceSheet["totalAssets"][0]
    x2 = balanceSheet["retainedEarnings"][0] / balanceSheet["totalAssets"][0]
    x3 = incomeStatement["ebit"][0] / balanceSheet["totalAssets"][0]
    x4 = stats["marketcap"] / balanceSheet["totalLiabilities"][0]
    return 6.56 * x1 + 3.26 * x2 + 6.72 * x3 + 1.05 * x4

In [ ]:
altmanZDoublePrime("aapl")

In [ ]:
import numpy as np

def altmanZDPImpliedRating( ticker ):
    '''
    Calculate the implied credit rating from a company's Altman Z" Score 
    
    ticker = string, user input for which to calculate the Z-score. Not case sensitive.
    '''
    adjZScore = 3.25 + altmanZDoublePrime( ticker )
    zMap = [ 8.15, 7.6, 7.3, 7., 6.85, 6.65, 6.4, 6.25, 5.85, 5.65, 5.25, 4.95, 4.75, 4.5, 4.15, 3.75, 3.2, 2.5, 1.75 ]
    scores = [ "AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB+", "BBB", "BBB-", "BB+", "BB", "BB-", "B+", "B", "B-", "CCC+", "CCC", "CCC-", "D" ] 
    return scores[ zMap.index( np.array( zMap )[ np.array( zMap ) < adjZScore ].max() ) ]

In [ ]:
altmanZDPImpliedRating("aapl")